In [1]:
import numpy as np
import xarray as xr
from pathlib import Path
from datetime import datetime
import utils_EOATS as util

In [2]:
def data_extraction_all_levels(DATA_ROOT, OUT_DIR, pres, keep_indices=None, chunk_size=125_000, chunk_index=None):
    """
    Extract T, S, and error for all pressurelevels in one pass.
    pres: 1D array of all standard pressure levels (root["pressure"][:])
    """
    OUT_DIR = Path(OUT_DIR)
    OUT_DIR.mkdir(parents=True, exist_ok=True)
    paths = np.load(OUT_DIR / "paths.npy")

    if keep_indices is None:
        keep_indices = np.arange(len(pres))
    keep_indices = np.asarray(keep_indices)
    n_keep = keep_indices.size
    print("n_keep:", n_keep)

    temp_arr = np.full((chunk_size, n_keep), np.nan, dtype=np.float32)
    sal_arr  = np.full((chunk_size, n_keep), np.nan, dtype=np.float32)
    p_err_arr = np.full((chunk_size, n_keep), np.nan, dtype=np.float32)
    t_err_arr = np.full((chunk_size, n_keep), np.nan, dtype=np.float32)
    s_err_arr = np.full((chunk_size, n_keep), np.nan, dtype=np.float32)

    pres_keep_vals = np.asarray(pres)[keep_indices]
    p_to_col = {round(float(p), 5): j for j, p in enumerate(pres_keep_vals)}
    start = datetime.now()
    DATA_ROOT = Path(DATA_ROOT)

    for i, rel_path in enumerate(paths[chunk_size*chunk_index:chunk_size*(chunk_index+1)]):
        path = DATA_ROOT / rel_path 

        with open(path, "rb", buffering=1024 * 1024) as f:
            for line in f:
                if not line:
                    continue
                if line.startswith(b"#") or line.startswith(b"pressure "):
                    continue

                cols = line.split(b",")
                if len(cols) < 6:
                    continue

                pres_b, temp_b, sali_b, perr_b, terr_b, serr_b = cols[:6]

                try:
                    p_line = round(float(pres_b), 5)
                except ValueError:
                    continue

                j = p_to_col.get(p_line)
                if j is None:
                    continue
                temp_arr[i, j] = util.safe_float(temp_b)
                sal_arr[i, j]  = util.safe_float(sali_b)
                p_err_arr[i, j] = util.safe_float(perr_b)
                t_err_arr[i, j] = util.safe_float(terr_b)
                s_err_arr[i, j] = util.safe_float(serr_b)
        if (i + 1) % 50_000 == 0:
            print(f"i={chunk_size*chunk_index+i+1}, ({datetime.now()}): {datetime.now() - start}", flush=True)
        if (i + 1) == chunk_size:
            break
            
    np.save(OUT_DIR / str("temp_arr_all_levels_chunk_"+str(chunk_index)+".npy"), temp_arr)
    np.save(OUT_DIR / str("sal_arr_all_levels_chunk_"+str(chunk_index)+".npy"),  sal_arr)
    np.save(OUT_DIR / str("p_err_arr_all_levels_chunk_"+str(chunk_index)+".npy"), p_err_arr)
    np.save(OUT_DIR / str("t_err_arr_all_levels_chunk_"+str(chunk_index)+".npy"), t_err_arr)
    np.save(OUT_DIR / str("s_err_arr_all_levels_chunk_"+str(chunk_index)+".npy"), s_err_arr)

    print("Saved to", OUT_DIR)
    return None

In [3]:
OUT_DIR = Path("/home/jovyan/EasyOneArgoTSLite/derived")

In [4]:
DATA_ROOT = Path("/home/jovyan/EasyOneArgoTSLite/EasyOneArgoTSLite_20251015T172450Z")

In [26]:
for chunk_index in range(0, 22):
    data_extraction_all_levels(DATA_ROOT, OUT_DIR, pres, chunk_size=chunk_size, chunk_index=chunk_index)

n_keep: 107
Saved to /home/jovyan/EasyOneArgoTSLite/derived


In [63]:
store = gcs.get_mapper("pangeo-argo-eke/data/EasyOneArgoTSLite_v01")
root = zarr.open_group(store, mode="r+")
pres = root["pressure"][:]
print(root.tree())

for chunk_index in range(21,22):
    
    T = np.load(OUT_DIR / str("temp_arr_all_levels_chunk_"+str(chunk_index)+".npy")) 
    S = np.load(OUT_DIR / str("sal_arr_all_levels_chunk_"+str(chunk_index)+".npy"))
    Perr = np.load(OUT_DIR / str("p_err_arr_all_levels_chunk_"+str(chunk_index)+".npy"))
    Terr = np.load(OUT_DIR / str("t_err_arr_all_levels_chunk_"+str(chunk_index)+".npy"))
    Serr = np.load(OUT_DIR / str("s_err_arr_all_levels_chunk_"+str(chunk_index)+".npy"))
    print(str(chunk_index)+": T.shape =", T.shape,"     ", chunk_size*chunk_index,":",chunk_size*(chunk_index+1))
    root["temperature"][chunk_size*chunk_index:chunk_size*(chunk_index+1), :]       = T[:26963,:]
    root["salinity"][chunk_size*chunk_index:chunk_size*(chunk_index+1), :]          = S[:26963,:]
    root["pressure_error"][chunk_size*chunk_index:chunk_size*(chunk_index+1), :]    = Perr[:26963,:]
    root["temperature_error"][chunk_size*chunk_index:chunk_size*(chunk_index+1), :] = Terr[:26963,:]
    root["salinity_error"][chunk_size*chunk_index:chunk_size*(chunk_index+1), :]    = Serr[:26963,:]
    del T
    del S
    del Perr
    del Terr
    del Serr
    

/
├── latitude (2651963,) float32
├── longitude (2651963,) float32
├── paths (2651963,) <U41
├── pressure (107,) float64
├── pressure_error (2651963, 107) float32
├── profilelocation_index (2651963,) int64
├── salinity (2651963, 107) float32
├── salinity_error (2651963, 107) float32
├── temperature (2651963, 107) float32
├── temperature_error (2651963, 107) float32
└── time (2651963,) float64


21: T.shape = (125000, 107)       2625000 : 2750000


# Rechunking:

In [5]:
import zarr
gcs = util.get_gcs(asynchronous=False)
store = gcs.get_mapper("pangeo-argo-eke/data/EasyOneArgoTSLite_v01")
root = zarr.open_group(store, mode="r+")
pres = root["pressure"][:]

In [6]:
ds = xr.open_zarr(store, consolidated=True, chunks={})

print(ds["temperature"].sizes)
print(ds["temperature"].shape)

Frozen({'profilelocation_index': 2651963, 'pressure': 107})
(2651963, 107)


In [17]:
profile_dim, pressure_dim = ds["temperature"].sizes  
ds_rechunked = ds.chunk(
    {
        profile_dim: -1, 
        pressure_dim: 1,
    }
)
new_store = gcs.get_mapper("pangeo-argo-eke/data/EasyOneArgoTSLite_v01_Spaghetti")
twod_vars = [
    "pressure_error",
    "salinity",
    "salinity_error",
    "temperature",
    "temperature_error",
]

oned_profile_vars = [
    v for v in ds.data_vars
    if ds[v].dims == (profile_dim,) and v != "time"
]

encoding = {
    **{v: {"chunks": (ds.sizes[profile_dim], 1)} for v in twod_vars},
    **{v: {"chunks": (ds.sizes[profile_dim],)} for v in oned_profile_vars},
    "pressure": {"chunks": (ds.sizes[pressure_dim],)},
}
time_encoding = ds["time"].encoding.copy()
time_encoding["chunks"] = (ds.sizes[profile_dim],)
del time_encoding["preferred_chunks"]
encoding["time"] = time_encoding

In [18]:
encoding

{'pressure_error': {'chunks': (2651963, 1)},
 'salinity': {'chunks': (2651963, 1)},
 'salinity_error': {'chunks': (2651963, 1)},
 'temperature': {'chunks': (2651963, 1)},
 'temperature_error': {'chunks': (2651963, 1)},
 'latitude': {'chunks': (2651963,)},
 'longitude': {'chunks': (2651963,)},
 'paths': {'chunks': (2651963,)},
 'pressure': {'chunks': (107,)},
 'time': {'chunks': (2651963,),
  'compressors': (ZstdCodec(level=0, checksum=False),),
  'filters': (),
  'shards': None,
  'serializer': BytesCodec(endian=<Endian.little: 'little'>),
  '_FillValue': nan,
  'units': 'days since 1970-01-01 00:00:00',
  'dtype': dtype('<f8')}}

In [23]:
%%time
ds_rechunked.to_zarr(new_store, mode="w", encoding=encoding)

/srv/conda/envs/notebook/lib/python3.12/site-packages/zarr/core/dtype/npy/string.py:248: UnstableSpecificationWarning: The data type (FixedLengthUTF32(length=41, endianness='little')) does not have a Zarr V3 specification. That means that the representation of arrays saved with this data type may change without warning in a future version of Zarr Python. Arrays stored with this data type may be unreadable by other Zarr libraries. Use this data type at your own risk! Check https://github.com/zarr-developers/zarr-extensions/tree/main/data-types for the status of data type specifications for Zarr V3.
  v3_unstable_dtype_warning(self)
/srv/conda/envs/notebook/lib/python3.12/site-packages/zarr/api/asynchronous.py:244: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


CPU times: user 24.3 s, sys: 9.54 s, total: 33.8 s
Wall time: 2min 5s


In [24]:
root_new = zarr.open_group(new_store, mode="r+")

In [25]:
from datetime import datetime
root_new.attrs["initialized_date"] = str(datetime.now())
root_new.attrs["filename"] = new_store.root

/srv/conda/envs/notebook/lib/python3.12/site-packages/zarr/core/dtype/npy/string.py:248: UnstableSpecificationWarning: The data type (FixedLengthUTF32(length=41, endianness='little')) does not have a Zarr V3 specification. That means that the representation of arrays saved with this data type may change without warning in a future version of Zarr Python. Arrays stored with this data type may be unreadable by other Zarr libraries. Use this data type at your own risk! Check https://github.com/zarr-developers/zarr-extensions/tree/main/data-types for the status of data type specifications for Zarr V3.
  v3_unstable_dtype_warning(self)


In [29]:
ds_rc = xr.open_zarr(new_store)
ds_rc

<xarray.Dataset> Size: 6GB
Dimensions:                (profilelocation_index: 2651963, pressure: 107)
Coordinates:
  * profilelocation_index  (profilelocation_index) int64 21MB 0 1 ... 2651962
  * pressure               (pressure) float64 856B 2.0 5.0 ... 5.8e+03 6e+03
Data variables:
    latitude               (profilelocation_index) float32 11MB dask.array<chunksize=(2651963,), meta=np.ndarray>
    longitude              (profilelocation_index) float32 11MB dask.array<chunksize=(2651963,), meta=np.ndarray>
    paths                  (profilelocation_index) <U41 435MB dask.array<chunksize=(2651963,), meta=np.ndarray>
    pressure_error         (profilelocation_index, pressure) float32 1GB dask.array<chunksize=(2651963, 1), meta=np.ndarray>
    salinity               (profilelocation_index, pressure) float32 1GB dask.array<chunksize=(2651963, 1), meta=np.ndarray>
    salinity_error         (profilelocation_index, pressure) float32 1GB dask.array<chunksize=(2651963, 1), meta=np.ndarray>
    temperature            (profilelocation_index, pressure) float32 1GB dask.array<chunksize=(2651963, 1), meta=np.ndarray>
    temperature_error      (profilelocation_index, pressure) float32 1GB dask.array<chunksize=(2651963, 1), meta=np.ndarray>
    time                   (profilelocation_index) datetime64[ns] 21MB dask.array<chunksize=(2651963,), meta=np.ndarray>
Attributes:
    description:       EasyOneArgoTSLite 2025-10 version's variables, stored ...
    filename:          pangeo-argo-eke/data/EasyOneArgoTSLite_v01_Spaghetti
    initialized_date:  2025-12-02 19:54:50.155225